# ZenML: Open-source MLOps Framework for reproducible ML pipelines

![Test](_assets/Logo/zenml.svg)

<div class="alert alert-block alert-danger">
    <b>Note:</b> This lesson is still in progress and some commands may not work as described. Please expect an update until 20th April 2022. 
</div>

In [1]:
from absl import logging as absl_logging
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
absl_logging.set_verbosity(-10000)

Let's begin by initializing ZenML in our directory. We are going to use a local stack to begin with, for simplicity and then transition to other stacks. This can be achieved in code by executing the following block.

# Initialize ZenML

In [2]:
!rm -rf .zen
!zenml init

Registering default stack and user...
ZenML repository initialized at /Users/felix/code/zenbytes.
⠋ Initializing ZenML repository at /Users/felix/code/zenbytes.
⠋ Initializing ZenML repository at /Users/felix/code/zenbytes.

The local active profile was initialized to 'target' and the local active stack 
to 'default'. This local configuration will only take effect when you're running
ZenML from the initialized repository root, or from a subdirectory. For more 
information on profile and stack configuration, please visit 
https://docs.zenml.io.


# Install integrations

ZenML handles integrations natively, to avoid dependency conflicts, so make sure to use the following command to install the integrations required for this lesson.

![All](_assets/integrations_all.png "All")

![All](_assets/seldon.png "Seldon Model Deployer.png")

In [3]:
!zenml integration install kubeflow seldon s3 aws -f

⠼ Installing integrations...Requirement already satisfied: kfp==1.8.9 in /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/lib/python3.8/site-packages (1.8.9)
⠴ Installing integrations...Requirement already satisfied: wrapt>=1.10.10 in /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/lib/python3.8/site-packages (from aiobotocore~=2.2.0->s3fs==2022.3.0) (1.14.0)
⠦ Installing integrations...Requirement already satisfied: pyrsistent>=0.14.0 in /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/lib/python3.8/site-packages (from jsonschema<4,>=3.0.1->kfp==1.8.9) (0.18.1)
⠧ Installing integrations...Requirement already satisfied: oauthlib>=3.0.0 in /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/lib/python3.8/site-packages (from requests-oauthlib->kubernetes==18.20.0) (3.2.0)
⠇ Installing integrations...Requirement already satisfied: google-crc32c<2.0dev,>=1.0 in /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/lib/python3.8/site-packages (from google-resumable-media<3.0dev,>=1.3.0->googl

# The Concept of MLOps Stacks

The ZenML stack is a concept that describes the union of Metadata Store, Artifact Store and Orchestrator that will be used for all pipeline runs. When you get started with zenml you start off with a default local stack.

In [4]:
!zenml stack list

Registering default stack and user...
Running with active profile: 'target' (local)
┏━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┓
┃        │ STACK   │         │         │         │         │         │         ┃
┃ ACTIVE │ NAME    │ ARTIFA… │ METADA… │ ORCHES… │ CONTAI… │ MODEL_… │ SECRET… ┃
┠────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┨
┃   👉   │ default │ default │ default │ default │         │         │         ┃
┃        │ new_st… │ s3_sto… │ kubefl… │ eks_or… │ ecr_re… │ eks_se… │ aws_se… ┃
┗━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┛


## The Local Stack

You can imagine the local stack to look like this. Within the diagram we show how a generic pipeline interacts with the local stack.

![LocalStack](_assets/localstack.png "LocalStack")

## The Kubeflow Pipelines stack

We will now use the Kubeflow integration to extend the concept of stacks

Now we want to transition to a kubeflow stack that will look a little bit like this. Note that for kubeflow pipelines we also need a registry where the docker images for each step are registered. 

![KubeflowStack](_assets/aws_stack_seldon.png "KubeflowStack")

But we have good news! You barely have to do anything to transition.

# Transitioning to Production with Kubeflow on AWS

There are two steps to follow in order to continue.

- Set up the necessary cloud resources on the provider of your choice
- Configure ZenML with a new stack to be able to communicate with these resources

## Set up using the cloud guide

In order to continue, it is best to follow the updated cloud guide for ZenML found [here](https://docs.zenml.io/features/guide-aws-gcp-azure). Please return after finishing the `pre-requisites` section.

It is recommended you use AWS as your cloud provider to follow along the lesson. However, if you were to select GCP or Azure, it should not so hard to actually modify the below commands to work accordingly.

You will also need Seldon Core to be installed in the same Kubernetes cluster as Kubeflow. Some brief instructions on how to install Seldon Core in AWS EKS can be found [here](https://github.com/zenml-io/zenml/tree/main/examples/seldon_deployment#installing-seldon-core-eg-in-an-eks-cluster). It is also advisable to read the official [Seldon Core documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

## Create your AWS Kubeflow Stack

Now we can configure a new stack that points to your newly created resources on the cloud

If you remember from the main README, Kubernetes and Docker are a pre-requisite to this part of the guide. Please make sure you have them installed. You also need to install the [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) to move forward.

In [5]:
# Replace the following with your own configuration. Use the below as exemplary.

KUBE_CONTEXT="zenml-eks"
AWS_EKS_CLUSTER="zenhacks-cluster"
AWS_REGION="us-east-1"
ECR_REGISTRY_NAME="715803424590.dkr.ecr.us-east-1.amazonaws.com"
S3_BUCKET_NAME="s3://zenbytes-bucket"
KUBEFLOW_NAMESPACE="kubeflow"

First, set up local access to the AWS EKS cluster and the AWS ECR registry.

In [6]:
# Point Docker to the ECR registry
!aws ecr get-login-password --region {AWS_REGION} | docker login --username AWS --password-stdin {ECR_REGISTRY_NAME}

# Create a Kubernetes configuration context that points to the EKS cluster
!aws eks --region {AWS_REGION} update-kubeconfig --name {AWS_EKS_CLUSTER} --alias {KUBE_CONTEXT}

Login Succeeded
Updated context zenml-eks in /Users/felix/.kube/config


Extract the base URL that will be used by Seldon Core to expose all model servers:

In [7]:
INGRESS_HOST = ! echo $(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].hostname}')
INGRESS_HOST[0]

'abb84c444c7804aa98fc8c097896479d-377673393.us-east-1.elb.amazonaws.com'

Finally, register the ZenML Stack:

In [8]:
# Register container registry
!zenml container-registry register ecr_registry --type=default --uri={ECR_REGISTRY_NAME}

# Register orchestrator (Kubeflow on AWS)
!zenml orchestrator register eks_orchestrator --type=kubeflow --kubernetes_context={KUBE_CONTEXT} --synchronous=True

# Register metadata store and artifact store
!zenml metadata-store register kubeflow_metadata_store --type=kubeflow
!zenml artifact-store register s3_store --type=s3 --path={S3_BUCKET_NAME}

# Register the Seldon Core model deployer (Seldon on AWS)
!zenml model-deployer register eks_seldon --type=seldon --kubernetes_context={KUBE_CONTEXT} --kubernetes_namespace={KUBEFLOW_NAMESPACE} --base_url=http://{INGRESS_HOST[0]} --secret=s3_store

# Register a secret manager
!zenml secrets-manager register aws_secret_manager --type=aws

# Register the aws_kubeflow_stack
!zenml stack register aws_kubeflow_stack -m kubeflow_metadata_store -a s3_store -o eks_orchestrator -c ecr_registry -d eks_seldon -x aws_secret_manager

Registering default stack and user...
Running with active profile: 'target' (local)
⠋ Registering container registry 'ecr_registry'...
⠋ Registering container registry 'ecr_registry'...

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/bin/zenml:8 in <module>    │
│                                                                              │
│   5 from zenml.cli.cli import cli                                            │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│   9                                                                          │
│                                                                              │
│ /Users/felix/.pyenv/versions/3.8.13/envs/zenbytes/lib/python3.8/site-package │
│ s

In [9]:
!zenml stack set aws_kubeflow_stack
!zenml stack describe

Registering default stack and user...
Running with active profile: 'target' (local)
Active stack set to: 'aws_kubeflow_stack'
⠋ Setting the active stack to 'aws_kubeflow_stack'...beflow_stack'...
Registering default stack and user...
Running with active profile: 'target' (local)
              Stack Configuration               
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME          ┃
┠────────────────────┼─────────────────────────┨
┃ ARTIFACT_STORE     │ s3_store                ┃
┠────────────────────┼─────────────────────────┨
┃ CONTAINER_REGISTRY │ ecr_registry            ┃
┠────────────────────┼─────────────────────────┨
┃ METADATA_STORE     │ kubeflow_metadata_store ┃
┠────────────────────┼─────────────────────────┨
┃ MODEL_DEPLOYER     │ eks_seldon              ┃
┠────────────────────┼─────────────────────────┨
┃ ORCHESTRATOR       │ eks_orchestrator        ┃
┠────────────────────┼─────────────────────────┨
┃ SECRETS_MANAGER    │ aws_secret_

Next, we need to set up a ZenML Secret to give Seldon Core access to the AWS S3 artifact store in the configured namespace, by running the `zenml secret register` command.

NOTE: this is based on the assumption that Seldon Core is running in an EKS cluster that already has IAM access enabled and doesn't need any explicit AWS credentials. For more information on setting up ZenML secrets for Seldon Core, please see the [Managing Seldon Core Credentials](https://github.com/zenml-io/zenml/blob/main/examples/seldon_deployment/README.md#managing-seldon-core-credentials) section in our [Seldon Core Continuous Deployment Example](https://github.com/zenml-io/zenml/blob/main/examples/seldon_deployment/README.md).

For the IAM access case, you can run this command to create the secret:

`zenml secret register -s seldon_s3 s3_store`

, and only set the `rclone_config_s3_env_auth` key to `True`. However, we cannot do this in the Jupyter Notebook, because interactive CLI commands are not supported, so we'll do it programmatically:

In [10]:
from zenml.repository import Repository
from zenml.integrations.seldon.secret_schemas import SeldonS3SecretSchema

secrets_manager = Repository().active_stack.secrets_manager
secret = SeldonS3SecretSchema(
    name = "s3_store",
    rclone_config_s3_env_auth = True
)
try:
    secrets_manager.get_secret("s3_store")
except RuntimeError:
    secrets_manager.register_secret(secret)

!zenml secret get s3_store

Registering default stack and user...


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Registering default stack and user...
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
┏━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━┓
┃ SECRET_NAME │ SECRET_KEY                │ SECRET_VALUE ┃
┠─────────────┼───────────────────────────┼──────────────┨
┃  s3_store   │ rclone_config_s3_type     │ s3           ┃
┃  s3_store   │ rclone_config_s3_provider │ aws          ┃
┃  s3_store   │ rclone_config_s3_env_auth │ True         ┃
┗━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━┛


In [11]:
!zenml stack up

Registering default stack and user...
Running with active profile: 'target' (local)
Provisioning resources for active stack 'aws_kubeflow_stack'.
Provisioning resources for stack 'aws_kubeflow_stack'.
Provisioning local Kubeflow Pipelines deployment...
Provisioned resources for KubeflowMetadataStore(type=metadata_store, flavor=kubeflow, name=kubeflow_metadata_store, uuid=002e6287-0be3-40cd-91c2-d47706c401cf, upgrade_migration_enabled=False, host=127.0.0.1, port=8081).
Resuming provisioned resources for stack aws_kubeflow_stack.
Started Kubeflow Pipelines UI daemon (check the daemon logs at /Users/felix/Library/Application Support/zenml/kubeflow/7c4fdf82-2a63-4436-b55e-687b0860ba21/kubeflow_daemon.log in case you're not able to view the UI). The Kubeflow Pipelines UI should now be accessible at http://localhost:8080/.
Resumed resources for KubeflowOrchestrator(type=orchestrator, flavor=kubeflow, name=eks_orchestrator, uuid=7c4fdf82-2a63-4436-b55e-687b0860ba21, custom_docker_base_image_n

The output will indicate the URL that we can access to view Kubeflow pipelines locally (e.g. [http://localhost:8080/](http://localhost:8080/)).

## Transition to Production (Run on the Cloud)

Once the stack is configured, all that is left to do is to set it active and to run a pipeline. Note that the code itself DOES NOT need to change, only the active stack.

ZenML will detect that the stack has changed, and instead of running your pipeline locally, will build a Docker Image, push it to the container registry with your requirements, and deploy the pipeline with that image on Kubeflow Pipelines. This whole process is usually very painful but simplified with ZenML, and is completely customizable.

For now, try it out! It might take a few minutes to build and push the image, but after that you'd see your pipeline in the cloud!

<div class="alert alert-block alert-info">
    <b>Note:</b> Currently running pipelines defined within a jupyter notebook cell is
    not supported. To get around this you can run the train pipeline within this repo. 
</div>

In [14]:
# Let's train within kubeflow pipelines - this will deploy the pipeline
!python src/run.py --deploy # --interval-second=300

Registering default stack and user...
Creating run for pipeline: `continuous_deployment_pipeline`
Cache disabled for pipeline `continuous_deployment_pipeline`
Registered stack component with type 'artifact_store' and name 's3_store'.
Registered stack component with type 'container_registry' and name 'ecr_registry'.
Registered stack component with type 'metadata_store' and name 'kubeflow_metadata_store'.
Registered stack component with type 'model_deployer' and name 'eks_seldon'.
Registered stack component with type 'orchestrator' and name 'eks_orchestrator'.
Registered stack component with type 'secrets_manager' and name 'aws_secret_manager'.
Registered stack with name 'aws_kubeflow_stack'.
No explicit dockerignore specified and no file called .dockerignore exists at the build context root (/Users/felix/code/zenbytes). Creating docker build context with all files inside the build context root directory.
Building docker image '715803424590.dkr.ecr.us-east-1.amazonaws.com/zenml-kubeflow:

In [15]:
!zenml served-models list

Registering default stack and user...
┏━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┓
┃ STATUS │ UUID             │ PIPELINE_NAME    │ PIPELINE_STEP_N… │ MODEL_NAME ┃
┠────────┼──────────────────┼──────────────────┼──────────────────┼────────────┨
┃   ✅   │ 01f049c8-c713-4… │ continuous_depl… │ seldon_model_de… │ model      ┃
┗━━━━━━━━┷━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━┛


And that's it you have successfully transitioned from local to production by simply switching you ZenML stack. This is just scratching the surface!

Next up, more about stacks, running pipelines on a schedule, and much more coming soon!

## Cleanup

Once you are done running pipelines with the AWS stack, you can run the following command to stop the Seldon Core model server and the local daemons:

In [16]:
!zenml served-models delete 01f049c8-c713-4e35-96c4-9f3738440762
!zenml stack down -f

Registering default stack and user...
⠋ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠙ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠹ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠸ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠼ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠴ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠦ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
model-serving, flavor: seldon)'.
⠧ Stopping service 
'SeldonDeploymentService[01f049c8-c713-4e35-96c4-9f3738440762] (type: 
mod